In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import wrds
import psycopg2 
import matplotlib.pyplot as plt
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
from scipy import stats
import pickle

In [2]:
crsp = pd.read_pickle('crsp.pkl')

In [3]:
# jdate = date + MonthEnd(0)
# retadj is the return that add delisting return
crsp

AttributeError: 'DataFrame' object has no attribute '_data'

In [4]:
# iclink is the link table between crsp and ibes
iclink = pd.read_pickle('iclink.pkl')
iclink_temp = pd.merge(crsp, iclink, on = 'permno', how = 'left')
iclink_temp

,permno,permco,date,shrcd,exchcd,ret,retx,prc,cfacshr,jdate,retadj,ticker,cname,comnam,name_ratio,score
0,63845,4,1986-01-31,11,1,0.071429,0.071429,35.625000,1.00,1986-01-31,0.071429,AAL,ALXNDR & ALXNDR,ALEXANDER & ALEXANDER SVCS INC,50.0,1.0
1,15580,5,1986-01-31,11,3,-0.068966,-0.068966,13.500000,1.25,1986-01-31,-0.068966,AAME,ATLAN AMERN,ATLANTIC AMERICAN CORP,67.0,0.0
2,14593,7,1986-01-31,11,3,0.051136,0.051136,23.125000,56.00,1986-01-31,0.051136,AAPL,APPLE,APPLE INC,100.0,0.0
3,11843,17,1986-01-31,10,3,-0.024272,-0.024272,25.125000,1.00,1986-01-31,-0.024272,ABEV,ALLECO INC,ALLECO INC,100.0,0.0
4,12539,18,1986-01-31,11,3,0.053097,0.053097,14.875000,2.00,1986-01-31,0.053097,ABIG,AMERN BKR INS,AMERICAN BANKERS INS GROUP INC,60.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2168844,19137,56971,2019-12-31,11,3,0.000000,NaN,4.100000,1.00,2019-12-31,0.000000,GRWG,GROWGENERATION,GROWGENERATION CORP,100.0,2.0
2168845,19139,56975,2019-12-31,11,3,0.000000,NaN,17.838600,1.00,2019-12-31,0.000000,LMPX,LMP AUTOM HLDG,LMP AUTOMOTIVE HOLDINGS INC,68.0,2.0
2168846,19142,56978,2019-12-31,11,3,0.000000,NaN,16.600000,1.00,2019-12-31,0.000000,MNPR,MONOPAR,MONOPAR THERAPEUTICS INC,100.0,2.0
2168847,19143,56979,2019-12-31,11,3,0.000000,NaN,-3.765000,1.00,2019-12-31,0.000000,028K,NEWGIOCO GROUP,NEWGIOCO GROUP INC,100.0,0.0


In [5]:
# score:
# - 0: BEST match: using (cusip, cusip dates and company names)       
#          or (exchange ticker, company names and 6-digit cusip)     
# - 1: Cusips and cusip dates match but company names do not match    
iclink_temp = iclink_temp[(iclink_temp.score == 1.0) | (iclink_temp.score == 0.0)]

In [6]:
# this is the actuals file
act = pd.read_csv('act.csv')
act

,TICKER,CUSIP,OFTIC,PENDS,MEASURE,PDICITY,ANNDATS,VALUE
0,0000,NaN,NaN,20121231,EPS,ANN,20140214,NaN
1,0000,NaN,NaN,20121231,EPS,QTR,20140214,NaN
2,0000,NaN,NaN,20130131,EPS,QTR,20140214,NaN
3,0000,NaN,NaN,20130331,EPS,QTR,20140214,NaN
4,0000,NaN,NaN,20130430,EPS,QTR,20140214,NaN
...,...,...,...,...,...,...,...,...
1031111,ZYNX,98986M10,ZYXI,20190930,EPS,QTR,20191029,0.06
1031112,ZYNX,98986M10,ZYXI,20191231,EPS,ANN,20200227,0.28
1031113,ZYNX,98986M10,ZYXI,20191231,EPS,QTR,20200227,0.09
1031114,ZYNX,98986M10,ZYXI,20200331,EPS,QTR,20200428,0.09


In [7]:
act = act.dropna(subset = ['VALUE'])
act = act.rename(str.lower, axis = 'columns')

In [8]:
act['pends'] = pd.to_datetime(act['pends'], format = '%Y%m%d')
act['anndats'] = pd.to_datetime(act['anndats'], format = '%Y%m%d')
act['jdate'] = act['anndats'] + MonthEnd(0)

In [9]:
act = act.rename(columns = {'value': 'eps'})
act = act[act['pdicity'] == 'ANN']

In [10]:
# adjust the actual eps
# adjustactual_t = actual_t * cfacshr_t-1 / cfacshr_t
ibact_crsp = pd.merge(iclink_temp, act[['ticker', 'pends', 'anndats', 'eps', 'jdate']], on = ['ticker', 'jdate'], how = 'left')
cfac = ibact_crsp[['permno', 'cfacshr', 'jdate', 'ticker']]
cfac = cfac.rename(columns = {'cfacshr': 'cfacshr_adj', 'jdate': 'pends'})
ibact_crsp2 = pd.merge(ibact_crsp, cfac, on = ['permno', 'ticker', 'pends'], how = 'left')
ibact_crsp2

,permno,permco,date,shrcd,exchcd,ret,retx,prc,cfacshr,jdate,retadj,ticker,cname,comnam,name_ratio,score,pends,anndats,eps,cfacshr_adj
0,63845,4,1986-01-31,11,1,0.071429,0.071429,35.625,1.00,1986-01-31,0.071429,AAL,ALXNDR & ALXNDR,ALEXANDER & ALEXANDER SVCS INC,50.0,1.0,NaT,NaT,NaN,NaN
1,15580,5,1986-01-31,11,3,-0.068966,-0.068966,13.500,1.25,1986-01-31,-0.068966,AAME,ATLAN AMERN,ATLANTIC AMERICAN CORP,67.0,0.0,NaT,NaT,NaN,NaN
2,14593,7,1986-01-31,11,3,0.051136,0.051136,23.125,56.00,1986-01-31,0.051136,AAPL,APPLE,APPLE INC,100.0,0.0,NaT,NaT,NaN,NaN
3,11843,17,1986-01-31,10,3,-0.024272,-0.024272,25.125,1.00,1986-01-31,-0.024272,ABEV,ALLECO INC,ALLECO INC,100.0,0.0,NaT,NaT,NaN,NaN
4,12539,18,1986-01-31,11,3,0.053097,0.053097,14.875,2.00,1986-01-31,0.053097,ABIG,AMERN BKR INS,AMERICAN BANKERS INS GROUP INC,60.0,1.0,NaT,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1971076,19072,56947,2019-12-31,11,3,0.050279,0.050279,13.160,1.00,2019-12-31,0.050279,GRTX,GALERA,GALERA THERAPEUTICS INC,100.0,0.0,NaT,NaT,NaN,NaN
1971077,19081,56955,2019-12-31,11,3,0.465517,0.465517,25.500,1.00,2019-12-31,0.465517,SITM,SITIME,SITIME CORP,100.0,0.0,NaT,NaT,NaN,NaN
1971078,19082,56957,2019-12-31,11,3,-0.061594,-0.061594,12.950,1.00,2019-12-31,-0.061594,02R5,TELA BIO,TELA BIO INC,100.0,0.0,NaT,NaT,NaN,NaN
1971079,19098,56962,2019-12-31,11,1,0.000000,NaN,22.680,1.00,2019-12-31,0.000000,HESM,HESS MIDSTREAM,HESS MIDSTREAM L P,100.0,0.0,NaT,NaT,NaN,NaN


In [11]:
ibact_crsp2['eps_adj'] = ibact_crsp2['eps']*ibact_crsp2['cfacshr_adj']/ibact_crsp2['cfacshr']

In [12]:
ibact_crsp2['eps_adj'].isnull().value_counts()

True     1836261
False     134820
Name: eps_adj, dtype: int64

In [13]:
realeps = ibact_crsp2[ibact_crsp2['eps_adj'].isnull() == False]

In [14]:
with open('realeps.pkl', 'wb') as f:
    pickle.dump(realeps, f)

In [15]:
# This is the forecast file
fc = pd.read_csv('sum.csv')
fc

,TICKER,CUSIP,OFTIC,STATPERS,MEASURE,FPI,MEDEST,MEANEST,STDEV,HIGHEST,LOWEST,FPEDATS
0,0000,87482X10,TLMR,20140417,EPS,6,0.07,0.08,0.01,0.10,0.07,20140331
1,0000,87482X10,TLMR,20140515,EPS,6,0.13,0.13,0.01,0.15,0.12,20140630
2,0000,87482X10,TLMR,20140619,EPS,6,0.13,0.13,0.01,0.15,0.12,20140630
3,0000,87482X10,TLMR,20140717,EPS,6,0.13,0.13,0.01,0.15,0.12,20140630
4,0000,87482X10,TLMR,20140417,EPS,7,0.11,0.12,0.02,0.15,0.10,20140630
...,...,...,...,...,...,...,...,...,...,...,...,...
7871095,ZYNX,98986M10,ZYXI,20200416,EPS,2,0.66,0.60,0.10,0.66,0.48,20211231
7871096,ZYNX,98986M10,ZYXI,20200514,EPS,2,0.71,0.71,0.20,0.97,0.48,20211231
7871097,ZYNX,98986M10,ZYXI,20200618,EPS,2,0.80,0.78,0.18,0.97,0.55,20211231
7871098,ZYNX,98986M10,ZYXI,20200716,EPS,2,0.67,0.66,0.21,0.89,0.43,20211231


In [16]:
fc = fc.rename(str.lower, axis = 'columns')
fc['statpers'] = pd.to_datetime(fc['statpers'], format = '%Y%m%d')
fc['fpedats'] = pd.to_datetime(fc['fpedats'], format = '%Y%m%d')
fc = fc.rename(columns = {'fpedats': 'pends'})
fc.head()

,ticker,cusip,oftic,statpers,measure,fpi,medest,meanest,stdev,highest,lowest,pends
0,0000,87482X10,TLMR,2014-04-17,EPS,6,0.07,0.08,0.01,0.10,0.07,2014-03-31
1,0000,87482X10,TLMR,2014-05-15,EPS,6,0.13,0.13,0.01,0.15,0.12,2014-06-30
2,0000,87482X10,TLMR,2014-06-19,EPS,6,0.13,0.13,0.01,0.15,0.12,2014-06-30
3,0000,87482X10,TLMR,2014-07-17,EPS,6,0.13,0.13,0.01,0.15,0.12,2014-06-30
4,0000,87482X10,TLMR,2014-04-17,EPS,7,0.11,0.12,0.02,0.15,0.10,2014-06-30


In [17]:
fpi1 = fc[fc.fpi == 1]
fpi2 = fc[fc.fpi == 2]

In [18]:
def rename_ibes(df, var_name):
    df = df.rename(columns={'medest': f'med_{var_name}',
                            'meanest': f'mean_{var_name}',
                            'stdev': f'std_{var_name}',
                           'highest': f'high_{var_name}',
                           'lowest': f'low_{var_name}'})
    return df

In [21]:
fpi1 = rename_ibes(fpi1, 'fpi1')
fpi2 = rename_ibes(fpi2, 'fpi2')
fpi1.head(12)

,ticker,cusip,oftic,statpers,measure,fpi,med_fpi1,mean_fpi1,std_fpi1,high_fpi1,low_fpi1,pends
12,0000,87482X10,TLMR,2014-04-17,EPS,1,0.51,0.52,0.03,0.56,0.50,2014-12-31
13,0000,87482X10,TLMR,2014-05-15,EPS,1,0.58,0.56,0.04,0.58,0.50,2014-12-31
14,0000,87482X10,TLMR,2014-06-19,EPS,1,0.58,0.56,0.04,0.59,0.50,2014-12-31
15,0000,87482X10,TLMR,2014-07-17,EPS,1,0.58,0.56,0.05,0.59,0.50,2014-12-31
16,0000,87482X10,TLMR,2014-08-14,EPS,1,1.17,1.18,0.06,1.24,1.09,2014-12-31
17,0000,87482X10,TLMR,2014-09-18,EPS,1,1.17,1.17,0.06,1.24,1.09,2014-12-31
18,0000,87482X10,TLMR,2014-10-16,EPS,1,1.17,1.17,0.06,1.24,1.09,2014-12-31
19,0000,87482X10,TLMR,2014-11-20,EPS,1,1.20,1.19,0.03,1.22,1.14,2014-12-31
20,0000,87482X10,TLMR,2014-12-18,EPS,1,1.20,1.19,0.03,1.22,1.14,2014-12-31
21,0000,87482X10,TLMR,2015-01-15,EPS,1,1.20,1.19,0.03,1.22,1.14,2014-12-31


In [22]:
fpi1.shape

(1847081, 12)

In [23]:
fpi2.shape

(1723529, 12)

In [24]:
forecast = pd.merge(fpi1[['ticker', 'statpers', 'pends', 'mean_fpi1']], fpi2[['ticker', 'statpers', 'pends', 'mean_fpi2']],
                   on = ['ticker', 'pends'], suffixes = ['_1', '_2'], how = 'inner')
forecast.shape

(17684554, 6)

In [25]:
forecast

,ticker,statpers_1,pends,mean_fpi1,statpers_2,mean_fpi2
0,0000,2015-02-19,2015-12-31,0.88,2014-04-17,0.85
1,0000,2015-02-19,2015-12-31,0.88,2014-05-15,0.87
2,0000,2015-02-19,2015-12-31,0.88,2014-06-19,0.88
3,0000,2015-02-19,2015-12-31,0.88,2014-07-17,0.88
4,0000,2015-02-19,2015-12-31,0.88,2014-08-14,0.90
...,...,...,...,...,...,...
17684549,ZYNX,2020-08-20,2020-12-31,0.33,2019-10-17,0.34
17684550,ZYNX,2020-08-20,2020-12-31,0.33,2019-11-14,0.34
17684551,ZYNX,2020-08-20,2020-12-31,0.33,2019-12-19,0.34
17684552,ZYNX,2020-08-20,2020-12-31,0.33,2020-01-16,0.34


In [27]:
# merge the forecasts and the actuals
fcac = pd.merge(realeps[['permno', 'ticker', 'pends', 'jdate', 'eps_adj']], forecast, on = ['ticker', 'pends'], how = 'inner')
fcac.head(15)

,permno,ticker,pends,jdate,eps_adj,statpers_1,mean_fpi1,statpers_2,mean_fpi2
0,77730,TYSN,1986-09-30,1986-11-30,1.1790,1986-02-20,2.05,1986-01-16,1.97
1,77730,TYSN,1986-09-30,1986-11-30,1.1790,1986-03-20,2.06,1986-01-16,1.97
2,77730,TYSN,1986-09-30,1986-11-30,1.1790,1986-04-17,1.02,1986-01-16,1.97
3,77730,TYSN,1986-09-30,1986-11-30,1.1790,1986-05-15,1.03,1986-01-16,1.97
4,77730,TYSN,1986-09-30,1986-11-30,1.1790,1986-06-19,1.05,1986-01-16,1.97
5,77730,TYSN,1986-09-30,1986-11-30,1.1790,1986-07-17,1.09,1986-01-16,1.97
6,77730,TYSN,1986-09-30,1986-11-30,1.1790,1986-08-14,1.09,1986-01-16,1.97
7,77730,TYSN,1986-09-30,1986-11-30,1.1790,1986-09-18,1.13,1986-01-16,1.97
8,77730,TYSN,1986-09-30,1986-11-30,1.1790,1986-10-16,1.17,1986-01-16,1.97
9,87725,EXPO,1986-09-30,1986-11-30,1.7708,1986-02-20,3.22,1986-01-16,3.30


In [35]:
fcac = fcac.rename(columns = {'eps_adj': 'pred_eps', 'jdate': 'pred_date'})
with open('fcac.pkl', 'wb') as f:
    pickle.dump(fcac, f)

In [30]:
realeps.sort_values(by = ['permno', 'pends', 'anndats']).head(15)

,permno,permco,date,shrcd,exchcd,ret,retx,prc,cfacshr,jdate,...,ticker,cname,comnam,name_ratio,score,pends,anndats,eps,cfacshr_adj,eps_adj
146595,10001,7953,1988-08-31,11,3,0.029126,0.029126,6.6250,3.0,1988-08-31,...,GFGC,GAS NAT,GAS NATURAL INC,64.0,1.0,1988-06-30,1988-08-31,0.550,3.0,0.550
203070,10001,7953,1989-08-31,11,3,0.275862,0.275862,9.2500,3.0,1989-08-31,...,GFGC,GAS NAT,GAS NATURAL INC,64.0,1.0,1989-06-30,1989-08-29,1.210,3.0,1.210
258977,10001,7953,1990-08-31,11,3,-0.050000,-0.050000,9.5000,3.0,1990-08-31,...,GFGC,GAS NAT,GAS NATURAL INC,64.0,1.0,1990-06-30,1990-08-29,1.090,3.0,1.090
319930,10001,7953,1991-09-30,11,3,0.013043,0.000000,11.5000,3.0,1991-09-30,...,GFGC,GAS NAT,GAS NATURAL INC,64.0,1.0,1991-06-30,1991-09-18,1.010,3.0,1.010
374180,10001,7953,1992-08-31,11,3,0.040000,0.040000,13.0000,3.0,1992-08-31,...,GFGC,GAS NAT,GAS NATURAL INC,64.0,1.0,1992-06-30,1992-08-26,0.770,3.0,0.770
442499,10001,7953,1993-09-30,11,3,0.066142,0.055118,16.7500,3.0,1993-09-30,...,GFGC,GAS NAT,GAS NATURAL INC,64.0,1.0,1993-06-30,1993-09-01,1.180,3.0,1.180
535853,10001,7953,1995-01-31,11,3,-0.031250,-0.031250,-7.7500,1.5,1995-01-31,...,GFGC,GAS NAT,GAS NATURAL INC,64.0,1.0,1994-06-30,1995-01-18,0.515,1.5,0.515
584530,10001,7953,1995-09-29,11,3,0.043750,0.031250,8.2500,1.5,1995-09-30,...,GFGC,GAS NAT,GAS NATURAL INC,64.0,1.0,1995-06-30,1995-09-14,0.680,1.5,0.680
729305,10001,7953,1997-07-31,11,3,0.045455,0.045455,8.6250,1.5,1997-07-31,...,GFGC,GAS NAT,GAS NATURAL INC,64.0,1.0,1996-06-30,1997-07-21,0.950,1.5,0.950
722465,10001,7953,1997-06-30,11,3,-0.030725,-0.043478,8.2500,1.5,1997-06-30,...,GFGC,GAS NAT,GAS NATURAL INC,64.0,1.0,1996-12-31,1997-06-03,0.650,1.5,0.650


In [36]:
# construct the 1-year ahead dataset
real_forecast1 = pd.merge_asof(realeps[['permno', 'ticker', 'anndats', 'eps_adj']].sort_values('anndats'), fcac[['permno', 'ticker', 'pred_date', 'pred_eps', 'statpers_1', 'mean_fpi1']].sort_values('statpers_1'), 
              left_on = 'anndats', right_on = 'statpers_1', by = ['permno', 'ticker'], direction = 'forward').sort_values(['permno', 'statpers_1'])

In [38]:
real_forecast1.head(20)

,permno,ticker,anndats,eps_adj,pred_date,pred_eps,statpers_1,mean_fpi1
5260,10001,GFGC,1988-08-31,0.550,1992-08-31,0.770,1991-09-19,1.13
8279,10001,GFGC,1989-08-29,1.210,1992-08-31,0.770,1991-09-19,1.13
11948,10001,GFGC,1990-08-29,1.090,1992-08-31,0.770,1991-09-19,1.13
15905,10001,GFGC,1991-09-18,1.010,1992-08-31,0.770,1991-09-19,1.13
19871,10001,GFGC,1992-08-26,0.770,1993-09-30,1.180,1992-09-17,1.13
24513,10001,GFGC,1993-09-01,1.180,1995-01-31,0.515,1993-09-16,1.33
30436,10001,GFGC,1995-01-18,0.515,1995-09-30,0.680,1995-01-19,0.61
34624,10001,GFGC,1995-09-14,0.680,1997-07-31,0.950,1995-09-14,0.70
45509,10001,GFGC,1997-06-03,0.650,1998-11-30,0.640,1997-06-19,0.63
45748,10001,GFGC,1997-07-21,0.950,1998-01-31,0.550,1997-08-14,0.62


In [39]:
# keep the latest eps
real_forecast1 = real_forecast1.sort_values(['permno', 'statpers_1', 'anndats']).drop_duplicates(subset = ['permno', 'statpers_1'], keep = 'last')

In [40]:
real_forecast1.head(10)

,permno,ticker,anndats,eps_adj,pred_date,pred_eps,statpers_1,mean_fpi1
15905,10001,GFGC,1991-09-18,1.010,1992-08-31,0.770,1991-09-19,1.13
19871,10001,GFGC,1992-08-26,0.770,1993-09-30,1.180,1992-09-17,1.13
24513,10001,GFGC,1993-09-01,1.180,1995-01-31,0.515,1993-09-16,1.33
30436,10001,GFGC,1995-01-18,0.515,1995-09-30,0.680,1995-01-19,0.61
34624,10001,GFGC,1995-09-14,0.680,1997-07-31,0.950,1995-09-14,0.70
45509,10001,GFGC,1997-06-03,0.650,1998-11-30,0.640,1997-06-19,0.63
45748,10001,GFGC,1997-07-21,0.950,1998-01-31,0.550,1997-08-14,0.62
47094,10001,GFGC,1998-01-07,0.550,1998-11-30,0.640,1998-01-15,0.61
52896,10001,GFGC,1998-11-30,0.640,1999-09-30,0.660,1998-12-17,0.63
58075,10001,GFGC,1999-09-10,0.660,2000-11-30,0.530,1999-09-16,0.65


In [41]:
real_forecast1 = real_forecast1[real_forecast1.statpers_1.isnull() == False]

In [42]:
# add price
real_fore_prc1 = pd.merge_asof(crsp[['permno', 'prc', 'jdate']], real_forecast1.sort_values('statpers_1'), left_on = 'jdate', right_on = 'statpers_1', 
                                by = 'permno', direction = 'forward')

In [43]:
real_fore_prc1 = real_fore_prc1[real_fore_prc1.statpers_1.isnull() == False].sort_values(['permno', 'jdate'])

In [44]:
real_fore_prc1['dif'] = real_fore_prc1['statpers_1'] - real_fore_prc1['jdate']
real_fore_prc1.head(10)

,permno,prc,jdate,ticker,anndats,eps_adj,pred_date,pred_eps,statpers_1,mean_fpi1,dif
4131,10001,-6.1250,1986-01-31,GFGC,1991-09-18,1.01,1992-08-31,0.77,1991-09-19,1.13,2057 days
9834,10001,-6.2500,1986-02-28,GFGC,1991-09-18,1.01,1992-08-31,0.77,1991-09-19,1.13,2029 days
15539,10001,-6.3125,1986-03-31,GFGC,1991-09-18,1.01,1992-08-31,0.77,1991-09-19,1.13,1998 days
21250,10001,-6.3750,1986-04-30,GFGC,1991-09-18,1.01,1992-08-31,0.77,1991-09-19,1.13,1968 days
26970,10001,-6.3125,1986-05-31,GFGC,1991-09-18,1.01,1992-08-31,0.77,1991-09-19,1.13,1937 days
32738,10001,-6.1250,1986-06-30,GFGC,1991-09-18,1.01,1992-08-31,0.77,1991-09-19,1.13,1907 days
38563,10001,-6.0625,1986-07-31,GFGC,1991-09-18,1.01,1992-08-31,0.77,1991-09-19,1.13,1876 days
44429,10001,-6.5000,1986-08-31,GFGC,1991-09-18,1.01,1992-08-31,0.77,1991-09-19,1.13,1845 days
50328,10001,6.3750,1986-09-30,GFGC,1991-09-18,1.01,1992-08-31,0.77,1991-09-19,1.13,1815 days
56257,10001,6.6250,1986-10-31,GFGC,1991-09-18,1.01,1992-08-31,0.77,1991-09-19,1.13,1784 days


In [45]:
# only use the recent year's data
real_fore_prc1 = real_fore_prc1[real_fore_prc1['dif'].dt.days <= 365].rename(columns = {'eps_adj': 'past_eps'}).drop(columns = 'dif')
real_fore_prc1.head(10)

,permno,prc,jdate,ticker,anndats,past_eps,pred_date,pred_eps,statpers_1,mean_fpi1
339947,10001,9.750,1990-09-30,GFGC,1991-09-18,1.01,1992-08-31,0.77,1991-09-19,1.13
345717,10001,-9.625,1990-10-31,GFGC,1991-09-18,1.01,1992-08-31,0.77,1991-09-19,1.13
351466,10001,9.625,1990-11-30,GFGC,1991-09-18,1.01,1992-08-31,0.77,1991-09-19,1.13
357185,10001,-9.500,1990-12-31,GFGC,1991-09-18,1.01,1992-08-31,0.77,1991-09-19,1.13
362888,10001,9.625,1991-01-31,GFGC,1991-09-18,1.01,1992-08-31,0.77,1991-09-19,1.13
368570,10001,9.750,1991-02-28,GFGC,1991-09-18,1.01,1992-08-31,0.77,1991-09-19,1.13
374240,10001,9.500,1991-03-31,GFGC,1991-09-18,1.01,1992-08-31,0.77,1991-09-19,1.13
379904,10001,-9.875,1991-04-30,GFGC,1991-09-18,1.01,1992-08-31,0.77,1991-09-19,1.13
385554,10001,-9.875,1991-05-31,GFGC,1991-09-18,1.01,1992-08-31,0.77,1991-09-19,1.13
391217,10001,10.500,1991-06-30,GFGC,1991-09-18,1.01,1992-08-31,0.77,1991-09-19,1.13


In [49]:
real_fore_prc1['prc'] = real_fore_prc1.prc.abs()

In [50]:
real_fore_prc1 = real_fore_prc1.set_index('statpers_1')
real_fore_prc1.head(15)

,permno,prc,jdate,ticker,anndats,past_eps,pred_date,pred_eps,mean_fpi1
statpers_1,,,,,,,,,
1991-09-19,10001,9.750,1990-09-30,GFGC,1991-09-18,1.01,1992-08-31,0.77,1.13
1991-09-19,10001,9.625,1990-10-31,GFGC,1991-09-18,1.01,1992-08-31,0.77,1.13
1991-09-19,10001,9.625,1990-11-30,GFGC,1991-09-18,1.01,1992-08-31,0.77,1.13
1991-09-19,10001,9.500,1990-12-31,GFGC,1991-09-18,1.01,1992-08-31,0.77,1.13
1991-09-19,10001,9.625,1991-01-31,GFGC,1991-09-18,1.01,1992-08-31,0.77,1.13
1991-09-19,10001,9.750,1991-02-28,GFGC,1991-09-18,1.01,1992-08-31,0.77,1.13
1991-09-19,10001,9.500,1991-03-31,GFGC,1991-09-18,1.01,1992-08-31,0.77,1.13
1991-09-19,10001,9.875,1991-04-30,GFGC,1991-09-18,1.01,1992-08-31,0.77,1.13
1991-09-19,10001,9.875,1991-05-31,GFGC,1991-09-18,1.01,1992-08-31,0.77,1.13


In [51]:
with open('real_fore_prc1.pkl', 'wb') as f:
    pickle.dump(real_fore_prc1, f)

In [52]:
# the constructioon of 2-year ahead data is the similar to 1-year ahead
real_forecast2 = pd.merge_asof(realeps[['permno', 'ticker', 'anndats', 'eps_adj']].sort_values('anndats'), fcac[['permno', 'ticker', 'pred_date', 'pred_eps', 'statpers_2', 'mean_fpi2']].sort_values('statpers_2'), 
              left_on = 'anndats', right_on = 'statpers_2', by = ['permno', 'ticker'], direction = 'forward').sort_values(['permno', 'statpers_2'])
real_forecast2.head(15)

,permno,ticker,anndats,eps_adj,pred_date,pred_eps,statpers_2,mean_fpi2
5260,10001,GFGC,1988-08-31,0.550,1992-08-31,0.770,1991-07-18,1.10
8279,10001,GFGC,1989-08-29,1.210,1992-08-31,0.770,1991-07-18,1.10
11948,10001,GFGC,1990-08-29,1.090,1992-08-31,0.770,1991-07-18,1.10
15905,10001,GFGC,1991-09-18,1.010,1993-09-30,1.180,1992-08-20,1.10
19871,10001,GFGC,1992-08-26,0.770,1995-01-31,0.515,1993-06-17,1.55
24513,10001,GFGC,1993-09-01,1.180,1995-09-30,0.680,1994-05-19,1.30
30436,10001,GFGC,1995-01-18,0.515,1997-07-31,0.950,1995-08-17,0.66
34624,10001,GFGC,1995-09-14,0.680,1998-01-31,0.550,1996-07-18,0.60
45509,10001,GFGC,1997-06-03,0.650,1998-11-30,0.640,1997-08-14,0.65
45748,10001,GFGC,1997-07-21,0.950,1998-11-30,0.640,1997-08-14,0.65


In [53]:
real_forecast2 = real_forecast2.sort_values(['permno', 'statpers_2', 'anndats']).drop_duplicates(subset = ['permno', 'statpers_2'], keep = 'last')
real_forecast2 = real_forecast2[real_forecast2.statpers_2.isnull() == False]

In [54]:
real_fore_prc2 = pd.merge_asof(crsp[['permno', 'prc', 'jdate']], real_forecast2.sort_values('statpers_2'), left_on = 'jdate', right_on = 'statpers_2', 
                                by = 'permno', direction = 'forward')

In [55]:
real_fore_prc2 = real_fore_prc2[real_fore_prc2.statpers_2.isnull() == False].sort_values(['permno', 'jdate'])

In [56]:
real_fore_prc2['dif'] = real_fore_prc2['statpers_2'] - real_fore_prc2['jdate']
real_fore_prc2.head(10)

,permno,prc,jdate,ticker,anndats,eps_adj,pred_date,pred_eps,statpers_2,mean_fpi2,dif
4131,10001,-6.1250,1986-01-31,GFGC,1990-08-29,1.09,1992-08-31,0.77,1991-07-18,1.1,1994 days
9834,10001,-6.2500,1986-02-28,GFGC,1990-08-29,1.09,1992-08-31,0.77,1991-07-18,1.1,1966 days
15539,10001,-6.3125,1986-03-31,GFGC,1990-08-29,1.09,1992-08-31,0.77,1991-07-18,1.1,1935 days
21250,10001,-6.3750,1986-04-30,GFGC,1990-08-29,1.09,1992-08-31,0.77,1991-07-18,1.1,1905 days
26970,10001,-6.3125,1986-05-31,GFGC,1990-08-29,1.09,1992-08-31,0.77,1991-07-18,1.1,1874 days
32738,10001,-6.1250,1986-06-30,GFGC,1990-08-29,1.09,1992-08-31,0.77,1991-07-18,1.1,1844 days
38563,10001,-6.0625,1986-07-31,GFGC,1990-08-29,1.09,1992-08-31,0.77,1991-07-18,1.1,1813 days
44429,10001,-6.5000,1986-08-31,GFGC,1990-08-29,1.09,1992-08-31,0.77,1991-07-18,1.1,1782 days
50328,10001,6.3750,1986-09-30,GFGC,1990-08-29,1.09,1992-08-31,0.77,1991-07-18,1.1,1752 days
56257,10001,6.6250,1986-10-31,GFGC,1990-08-29,1.09,1992-08-31,0.77,1991-07-18,1.1,1721 days


In [57]:
real_fore_prc2 = real_fore_prc2[real_fore_prc2['dif'].dt.days <= 730].rename(columns = {'eps_adj': 'past_eps'}).drop(columns = 'dif')

In [58]:
real_fore_prc2['prc'] = real_fore_prc2.prc.abs()

In [59]:
real_fore_prc2 = real_fore_prc2.set_index('statpers_2')
real_fore_prc2.head(15)

,permno,prc,jdate,ticker,anndats,past_eps,pred_date,pred_eps,mean_fpi2
statpers_2,,,,,,,,,
1991-07-18,10001,7.2500,1989-07-31,GFGC,1990-08-29,1.09,1992-08-31,0.77,1.1
1991-07-18,10001,9.2500,1989-08-31,GFGC,1990-08-29,1.09,1992-08-31,0.77,1.1
1991-07-18,10001,8.8750,1989-09-30,GFGC,1990-08-29,1.09,1992-08-31,0.77,1.1
1991-07-18,10001,9.5000,1989-10-31,GFGC,1990-08-29,1.09,1992-08-31,0.77,1.1
1991-07-18,10001,9.8750,1989-11-30,GFGC,1990-08-29,1.09,1992-08-31,0.77,1.1
1991-07-18,10001,10.1250,1989-12-31,GFGC,1990-08-29,1.09,1992-08-31,0.77,1.1
1991-07-18,10001,9.9375,1990-01-31,GFGC,1990-08-29,1.09,1992-08-31,0.77,1.1
1991-07-18,10001,9.8750,1990-02-28,GFGC,1990-08-29,1.09,1992-08-31,0.77,1.1
1991-07-18,10001,9.8750,1990-03-31,GFGC,1990-08-29,1.09,1992-08-31,0.77,1.1


In [61]:
with open('real_fore_prc2.pkl', 'wb') as f:
    pickle.dump(real_fore_prc2, f)